In [1]:
# ===============================================================
#  KAGGLE SETUP & IMPORTS
# ===============================================================
!pip install -q google-generativeai

import pandas as pd
import os
import time
from kaggle_secrets import UserSecretsClient # To get our API key securely

import google.generativeai as genai
# Import the specific types for safety settings
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# For our dynamic few-shot selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- Load API Key from Kaggle Secrets ---
try:
    user_secrets = UserSecretsClient()
    # Get the Google API key from the new secret
    secret_value = user_secrets.get_secret("GOOGLE_API_KEY") 
    genai.configure(api_key=secret_value)
    print("Google API key loaded successfully.")
    
    # Define robust safety settings
    safety_settings_config = [
        {
            "category": HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            "threshold": HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": HarmCategory.HARM_CATEGORY_HARASSMENT,
            "threshold": HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            "threshold": HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            "threshold": HarmBlockThreshold.BLOCK_NONE,
        },
    ]
    
    # Instantiate the Gemini model
    gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-05-20')
except Exception as e:
    print("Could not load Google API key or configure model. Make sure it's stored as a secret named 'GOOGLE_API_KEY'.")
    print(f"Error: {e}")
    # You might want to stop execution if the key isn't found
    # For a hackathon, we'll let it continue and fail on the API call
    gemini_model = None


# --- Load Data ---
# Adjust 'competition-folder-name' to the actual folder name in /kaggle/input/
DATA_PATH = "/kaggle/input/kenya-clinical-reasoning-challenge20250407/"
train_df = pd.read_csv(DATA_PATH + 'train.csv')
test_df = pd.read_csv(DATA_PATH + 'test.csv')
sample_submission_df = pd.read_csv(DATA_PATH + 'SampleSubmission.csv')

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Google API key loaded successfully.


Train data shape: (400, 12)
Test data shape: (100, 7)


In [2]:
# ===============================================================
#  DYNAMIC FEW-SHOT SELECTOR (Our "Secret Sauce")
# ===============================================================
# We will use TF-IDF to find the most similar prompts in the training data
# to use as examples in our LLM prompts. This is much better than static examples.

# Pre-process the training data for searching
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
train_prompt_vectors = vectorizer.fit_transform(train_df['Prompt'])

def get_most_similar_examples(query_prompt: str, top_n: int = 5) -> pd.DataFrame:
    """Finds the top_n most similar training examples to a given query prompt."""
    query_vector = vectorizer.transform([query_prompt])
    similarities = cosine_similarity(query_vector, train_prompt_vectors)
    # Get the indices of the top_n most similar prompts
    top_indices = np.argsort(similarities[0])[-top_n:][::-1]
    return train_df.iloc[top_indices]


In [3]:
# ===============================================================
#  SIMPLIFIED API CALL FUNCTION (Direct Inference)
# ===============================================================

def create_direct_response_prompt(prompt_text: str) -> str:
    """Creates a direct few-shot prompt for generating the clinician response."""
    # We'll use 2 examples, as this prompt is shorter than the old response prompt
    examples_df = get_most_similar_examples(prompt_text, top_n=5)
    
    prompt = "You are an expert Kenyan clinician providing practical guidance to a colleague. Your response MUST match the persona of the nurse in the prompt and the resources available at their facility. Structure your response with a summary, diagnosis, and a clear management plan. Emulate the style and reasoning from the examples below. Be concise and direct.\n\n--- EXAMPLES ---\n"
    
    for _, row in examples_df.iterrows():
        # We now show the direct Prompt -> Clinician mapping
        prompt += f"Vignette: \"{row['Prompt']}\"\n"
        prompt += f"Clinician Response: {row['Clinician']}\n\n"
        
    prompt += "--- TASK ---\n"
    prompt += f"Vignette: \"{prompt_text}\"\n"
    prompt += "Clinician Response:\n"
    return prompt

def generate_direct_response(prompt_text: str) -> str:
    """Uses a single, direct few-shot prompt to generate the clinician's response."""
    full_prompt = create_direct_response_prompt(prompt_text)
    
    try:
        generation_config = genai.GenerationConfig(temperature=0.4, max_output_tokens=65536)
        response = gemini_model.generate_content(full_prompt, generation_config=generation_config, safety_settings=safety_settings_config)

        if response.parts:
            return response.text.strip()
        else:
            finish_reason = response.candidates[0].finish_reason if response.candidates else 'UNKNOWN'
            return f"Error: No text generated. Finish Reason: {finish_reason}"

    except Exception as e:
        print(f"API Error in response generation: {e}")
        try:
            print(response.prompt_feedback)
        except:
            pass
        return "Error: Could not generate response."

In [4]:
# ===============================================================
#  MAIN EXECUTION SCRIPT
# ===============================================================
if gemini_model:
    results = []
    print("Starting prediction process with robust 'Direct-Inference' strategy...")
    
    RPM = 10
    requests_this_minute = 0
    start_time = time.time()
    
    for row in test_df.itertuples():
        print(f"Processing row {row.Index + 1}/{len(test_df)}: Master_Index = {row.Master_Index}")
        
        # Rate Limiting Check (now we can do more rows per minute!)
        if requests_this_minute >= RPM:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                sleep_time = 60.1 - elapsed_time
                print(f"  > Rate limit reached. Sleeping for {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
            start_time = time.time()
            requests_this_minute = 0

        prompt_text = row.Prompt
        
        # ONLY ONE API CALL PER ROW
        final_response = generate_direct_response(prompt_text)
        requests_this_minute += 1 # Increment counter
        
        print(f"  > Generated Response: {repr(final_response[:70])}...")
        
        results.append({'Master_Index': row.Master_Index, 'Clinician': final_response})
        print("-" * 20)
            
    print("Prediction process finished.")
    
    submission_df = pd.DataFrame(results)
    submission_df = submission_df[sample_submission_df.columns]
    submission_df.to_csv('submission.csv', index=False)
    
    print("Submission file 'submission.csv' has been created successfully!")
    print(submission_df.head())
else:
    print("Execution halted because Gemini model could not be initialized.")


Starting prediction process with robust 'Direct-Inference' strategy...
Processing row 1/100: Master_Index = ID_CUAOY


  > Generated Response: 'summary 24 yr old female with sharp right-sided nasal pain worsening o'...
--------------------
Processing row 2/100: Master_Index = ID_OGSAY


  > Generated Response: 'summary a 3 year old boy with a bean seed deep in the right nostril di'...
--------------------
Processing row 3/100: Master_Index = ID_TYHSA


  > Generated Response: 'summary 22 year old male presented with progressive lower limb weaknes'...
--------------------
Processing row 4/100: Master_Index = ID_CZXLD


  > Generated Response: 'summary a 6 year old girl presents with hx of sudden onset twitching f'...
--------------------
Processing row 5/100: Master_Index = ID_ZJQUQ


...

  > Generated Response: 'summary 35 year old male with 1 year infertility reduced libido and oc'...
--------------------
Processing row 99/100: Master_Index = ID_SNZBL


  > Generated Response: 'summary 48 year old post thyroidectomy patient with dizziness loss of '...
--------------------
Processing row 100/100: Master_Index = ID_KXLGG


  > Generated Response: 'summary a 58 year old patient presents with fever and chills and gener'...
--------------------
Prediction process finished.


Submission file 'submission.csv' has been created successfully!


  Master_Index                                          Clinician
0     ID_CUAOY  summary 24 yr old female with sharp right-side...
1     ID_OGSAY  summary a 3 year old boy with a bean seed deep...
2     ID_TYHSA  summary 22 year old male presented with progre...
3     ID_CZXLD  summary a 6 year old girl presents with hx of ...
4     ID_ZJQUQ  summary a 1 year old boy presented to mch clin...
